# 1. 하이퍼 파라미터 설정

In [ ]:
# train, validation 비율
test_size = 0.2

# 하이퍼파라미터
batch_size = 32
learning_rate = 0.001
num_epochs = 10

# 모델 인스턴스
input_size_x = 224
input_size_y = 224
input_size = input_size_x * input_size_y * 3
hidden_size = 128
num_classes = 2 # 한라봉, 감귤

# 데이터셋 경로
data_dir      = '/content/drive/MyDrive/jeju_data/Training/원천데이터'
test_data_dir = '/content/drive/MyDrive/jeju_data/Test/원천데이터'
model_save_path = '/content/drive/MyDrive/jeju_data/model.pth'

# 2. 필요한 도구 불러오기

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
# GPU가 있는 경우
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cuda


In [ ]:
def safe_remove_element(ls, value):
    try:
        ls.remove(value)
    except ValueError:
        pass
    return ls

class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.classes = safe_remove_element(os.listdir(data_dir), '.ipynb_checkpoints')
        self.class_to_dix = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        self.images = self._load_images()

    def _load_images(self):
        images = []
        for cls_name in self.classes:
            class_dir = os.path.join(self.data_dir, cls_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                images.append((img_path, cls_name))
        return images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path, cls_name = self.images[idx]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        label = self.class_to_dix[cls_name] # 클래스 이름을 인덱스로 변환
        return image, label


class SimpleModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleModel, self).__init__()
        self.flatten = nn.Flatten() # nn.view(-1, ~~~)
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


def train_epoch(model, data_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0

    for images, labels in tqdm(data_loader):
        optimizer.zero_grad()
        output = model(images.to(device))
        loss = criterion(output, labels.to(device))
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(data_loader.dataset)
    return epoch_loss


def evaluate_model(model, data_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in data_loader:
            labels = labels.to(device)
            output = model(images.to(device))
            loss = criterion(output, labels)
            running_loss += loss.item() * images.size(0)

            _, predicted = torch.max(output, dim=1)
            total += labels.size(0)
            correct += (predicted.to(device) == labels).sum().item()

    epoch_loss = running_loss / len(data_loader.dataset)
    accuarcy = correct/total
    return epoch_loss, accuarcy


def train_run(model, num_epochs, train_loader, eval_loader, optimizer, criterion):
    model.to(device)

    for epoch in range(num_epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer)
        val_loss, accuarcy = evaluate_model(model, eval_loader, criterion)

        print(f'Epoch [{epoch+1}/{num_epochs}] :',
              f'Train Loss: {train_loss},',
              f'Validation Loss: {val_loss},',
              f'accuarcy: {accuarcy}'
        )

transfrom = transforms.Compose([
    transforms.Resize((input_size_x, input_size_y)),
    transforms.ToTensor()
])

# 데이터 불러오기

In [ ]:
custom_dataset = CustomDataset(data_dir, transform=transfrom)

train_indces, eval_indices = train_test_split(list(range(len(custom_dataset))), test_size=test_size, random_state=0)

train_dataset = torch.utils.data.Subset(custom_dataset, train_indces)
eval_dataset = torch.utils.data.Subset(custom_dataset, eval_indices)
test_dataset = CustomDataset(test_data_dir, transform=transfrom)

train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
eval_data_loader = DataLoader(eval_dataset, batch_size=batch_size, shuffle=False)
test_data_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 모델 생성

In [ ]:
simple_model = SimpleModel(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(simple_model.parameters(), lr=learning_rate)

train_run(simple_model, num_epochs, train_data_loader, eval_data_loader, optimizer, criterion)
epoch_loss, accuarcy = evaluate_model(simple_model, test_data_loader, criterion)
print(f'Test Loss: {epoch_loss:.4f}, Test Accuarcy: {accuarcy:.4f}')

100%|██████████| 51/51 [01:01<00:00,  1.21s/it]


Epoch [1/10] : Train Loss: 6.1031109680520785, Validation Loss: 0.33397185550468445, accuarcy: 0.9276807980049875


100%|██████████| 51/51 [01:01<00:00,  1.21s/it]


Epoch [2/10] : Train Loss: 0.1566804214306513, Validation Loss: 0.3870053716431235, accuarcy: 0.8902743142144638


100%|██████████| 51/51 [01:01<00:00,  1.21s/it]


...

Epoch [8/10] : Train Loss: 0.01570349238954833, Validation Loss: 0.030882967714718317, accuarcy: 0.9925187032418953


100%|██████████| 51/51 [00:58<00:00,  1.15s/it]


Epoch [9/10] : Train Loss: 0.011721620592055632, Validation Loss: 0.02763200595775441, accuarcy: 0.9925187032418953


100%|██████████| 51/51 [00:57<00:00,  1.12s/it]


Epoch [10/10] : Train Loss: 0.01605386589601186, Validation Loss: 0.022285179110206756, accuarcy: 0.9950124688279302
Test Loss: 0.0212, Test Accuarcy: 0.9921


In [ ]:
# 모델 저장
torch.save(simple_model.state_dict(), model_save_path)

In [ ]:
# 모델 불러오기
simple_model = SimpleModel(input_size, hidden_size, num_classes)
simple_model.load_state_dict(torch.load(model_save_path))
simple_model.eval()